In [1]:
import pandas as pd

In [2]:
dtypes = {
    '관리건축물대장PK': 'string',
    '그룹 종류': 'Int64',
    '대장구분코드': 'string',
    '대장구분코드명': 'string',
    '대장종류코드': 'string',
    '대장종류코드명': 'string',
    '신구대장구분코드': 'string',
    '신구대장구분코드명': 'string',
    '대지위치': 'string',
    '도로명대지위치': 'string',
    '건물명': 'string',
    '시군구코드': 'string',
    '법정동코드': 'string',
    '대지구분코드': 'string',
    '번': 'string',
    '지': 'string',
    '새주소도로코드': 'string',
    '새주소법정동코드': 'string',
    '새주소지상지하코드': 'string',
    '새주소본번': 'Int64',
    '새주소부번': 'Int64',
    '대지면적(㎡)': 'Float64',
    '건축면적(㎡)': 'Float64',
    '건폐율(%)': 'Float64',
    '연면적(㎡)': 'Float64',
    '용적률산정연면적(㎡)': 'Float64',
    '용적률(%)': 'Float64',
    '주용도코드': 'string',
    '주용도코드명': 'string',
    '기타용도': 'string',
    '세대수(세대)': 'Int64',
    '가구수(가구)': 'Int64',
    '허가일': 'string',
    '착공일': 'string',
    '사용승인일': 'string',
    '호수(호)': 'Int64',
    '에너지효율등급': 'string',
    '에너지절감율': 'Float64',
    'EPI점수': 'Float64',
    '친환경건축물등급': 'string',
    '친환경건축물인증점수': 'Float64',
    '지능형건축물등급': 'string',
    '지능형건축물인증점수': 'Float64',
    '생성일자': 'string',
    '동명칭': 'string',
    '주부속구분코드': 'string',
    '주부속구분코드명': 'string',
    '구조코드': 'string',
    '구조코드명': 'string',
    '기타구조': 'string',
    '지붕코드': 'string',
    '지붕코드명': 'string',
    '기타지붕': 'string',
    '높이(m)': 'Float64',
    '지상층수': 'Int64',
    '지하층수': 'Int64',
    '승용승강기수': 'Int64',
    '비상용승강기수': 'Int64',
    '총동연면적(㎡)': 'Float64',
    '내진설계적용여부': 'string',
    '내진능력': 'string'
}

In [3]:
df = pd.read_csv(f'../output/국토부-총괄표제부-표제부-병합후-정제됨-25.csv', dtype=dtypes)
print(f'df : {len(df)}')

df : 613590


In [4]:
filtered_df = df[df['주용도코드명'].str.contains('단독주택|공동주택', na = False)]
print(f'filtered_df : {len(filtered_df)}')
filtered_df['주용도코드명'].unique()

filtered_df : 442615


<StringArray>
['단독주택', '공동주택']
Length: 2, dtype: string

In [7]:
filtered_df.to_excel('../output/건축물대장_주용도(단독주택,공동주택)_필터.xlsx', index=False)

Exception ignored in: <function ZipFile.__del__ at 0x104c2ea20>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1940, in __del__
    self.close()
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1957, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
